# [Module 9.0] Inferencde Pipeline 생성 로그 확인

Inout Pipeline의 Input, Output 약속


Preprocessor
CSV
    

In [1]:
import sagemaker
import pandas as pd
import numpy as np
import os
import time
import json
from time import strftime, gmtime

In [2]:
%store -r

#### 아래는 약 1분 정도가 소요 됩니다. 아래 셀의 [*] 의 표시가 [숫자] (에: [3])로 바뀔 때까지 기다려 주세요

In [3]:
from sagemaker.sklearn.estimator import SKLearn
sagemaker_session = sagemaker.Session()
from sagemaker import get_execution_role

role = get_execution_role()

script_path = 'log-preprocessing.py'
# instance_type = 'ml.m4.2xlarge'
instance_type = 'local'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    train_instance_type = instance_type
)
sklearn_preprocessor.fit({'train': s3_input_train})

Creating tmplhzdh25i_algo-1-qjia8_1 ... 
Attaching to tmplhzdh25i_algo-1-qjia8_12mdone
algo-1-qjia8_1  | 2020-08-13 08:38:20,280 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-qjia8_1  | 2020-08-13 08:38:20,283 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-qjia8_1  | 2020-08-13 08:38:20,291 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-qjia8_1  | 2020-08-13 08:38:20,397 sagemaker-containers INFO     Module log-preprocessing does not provide a setup.py. 
algo-1-qjia8_1  | Generating setup.py
algo-1-qjia8_1  | 2020-08-13 08:38:20,397 sagemaker-containers INFO     Generating setup.cfg
algo-1-qjia8_1  | 2020-08-13 08:38:20,397 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-qjia8_1  | 2020-08-13 08:38:20,397 sagemaker-containers INFO     Installing module with the following command:
algo-1-qjia8_1  | /miniconda3/bin/python -m pip install . 
algo-1-qjia8_1  |

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.


tmplhzdh25i_algo-1-qjia8_1 exited with code 0
Aborting on container exit...
===== Job Complete =====


## Feature Transfomer를 사용하여 전처리된 학습 및 검증 데이타 생성 

![Transformer_Train](img/Fig2.1.transformer_train.png)

### Preprocessed Training data (Feature) 만들기

#### 아래는 약 1분 정도가 소요 됩니다. 아래 셀의 [*] 의 표시가 [숫자] (에: [4])로 바뀔 때까지 기다려 주세요

In [4]:
# 아웃풋 경로 지정
transform_train_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-train-output')
instance_type = 'local'
# instance_type = 'ml.m4.2xlarge'

# scikit_learn_inferencee_model 이름으로 전처리 학습 모델 생성
# TRANSFORM_MODE의 환경 변수는 전처리 모드라는 것을 알려 줌.
    # 추론시에는 환경 변수를 TRANSFORM_MODE": "inverse-label-transform" 설정 함.
    # 위의 두개의 과정을 분리할 수 있으나, 한개의 소스를 (preprocessor.py)를 사용하기 위해서, 환경 변수를 통해서 구분함.
scikit_learn_inferencee_model = sklearn_preprocessor.create_model(
    env={'TRANSFORM_MODE': 'feature-transform'})
# scikit_learn_inferencee_model 에서 Train Transformer 생성
transformer_train = scikit_learn_inferencee_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_train_output_path,
    accept = 'text/csv')


# Preprocess training input
transformer_train.transform(s3_input_train.config['DataSource']['S3DataSource']['S3Uri'], 
                            content_type='text/csv')
print('Waiting for transform job: ' + transformer_train.latest_transform_job.job_name)
transformer_train.wait()
preprocessed_train_path = transformer_train.output_path + transformer_train.latest_transform_job.job_name
print(preprocessed_train_path)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


Attaching to tmpw3a5f0tw_algo-1-6l47c_1
algo-1-6l47c_1  | Processing /opt/ml/code
algo-1-6l47c_1  | Building wheels for collected packages: log-preprocessing
algo-1-6l47c_1  |   Building wheel for log-preprocessing (setup.py) ... done
algo-1-6l47c_1  |   Created wheel for log-preprocessing: filename=log_preprocessing-1.0.0-py2.py3-none-any.whl size=10222 sha256=221581449c41ac6bf5abd886a7b3b1ce07d98ec58b3f2f5b4ab304e257a3e04d
algo-1-6l47c_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-t4s6ur5m/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-6l47c_1  | Successfully built log-preprocessing
algo-1-6l47c_1  | Installing collected packages: log-preprocessing
algo-1-6l47c_1  | Successfully installed log-preprocessing-1.0.0
algo-1-6l47c_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternat

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.


algo-1-6l47c_1  | 2020-08-13 08:38:28,509 INFO - root - Input_fn, Mode: feature_transform
algo-1-6l47c_1  | 2020-08-13 08:38:28,518 INFO - root - predict_fn, Mode: feature_transform
algo-1-6l47c_1  | After trainsformation
algo-1-6l47c_1  |   (0, 0)	0.11941369588439606
algo-1-6l47c_1  |   (0, 1)	-0.5962380254245051
algo-1-6l47c_1  |   (0, 2)	1.744368057672484
algo-1-6l47c_1  |   (0, 3)	0.9789570533336895
algo-1-6l47c_1  |   (0, 4)	-0.028992907038264654
algo-1-6l47c_1  |   (0, 5)	-0.8931854019845896
algo-1-6l47c_1  |   (0, 6)	-0.8017032037830547
algo-1-6l47c_1  |   (0, 7)	-1.9825286353116254
algo-1-6l47c_1  |   (0, 8)	-1.5305589315744583
algo-1-6l47c_1  |   (0, 9)	-0.6008796483844902
algo-1-6l47c_1  |   (0, 10)	-0.44388847702445833
algo-1-6l47c_1  |   (0, 24)	1.0
algo-1-6l47c_1  |   (0, 65)	1.0
algo-1-6l47c_1  |   (0, 67)	1.0
algo-1-6l47c_1  |   (1, 0)	-1.852651636257096
algo-1-6l47c_1  |   (1, 1)	-0.5962380254245051
algo-1-6l47c_1  |   (1, 2)	0.1402835015166359
algo-1-6l47c_1  |   (1, 3

#### Training 전처리된 학습 파일 확인

In [5]:
print(preprocessed_train_path)

s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-13-08-38-2020-08-13-08-38-22-796


In [6]:
! aws s3 ls {preprocessed_train_path} --recursive

2020-08-13 08:38:29    1054526 sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-13-08-38-2020-08-13-08-38-22-796/train.csv.out


In [7]:
preprocessed_train_path_file = os.path.join (preprocessed_train_path, 'train.csv.out')
df_pre_train = pd.read_csv(preprocessed_train_path_file)
df_pre_train.head()


,0.0,0.11941369588439606,-0.5962380254245051,1.744368057672484,0.9789570533336895,-0.028992907038264654,-0.8931854019845896,-0.8017032037830547,-1.9825286353116254,-1.5305589315744583,...,0.0.48,0.0.49,0.0.50,0.0.51,0.0.52,0.0.53,1.0.1,0.0.54,1.0.2,0.0.55
0,0.0,-1.852652,-0.596238,0.140284,-0.310405,0.970689,-0.689888,0.146389,1.232901,0.124852,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,1.181295,-0.596238,1.835130,0.185503,0.030988,-0.639063,1.568529,-0.063643,-0.846802,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.776769,-0.596238,0.216227,0.334276,0.136954,1.393914,1.394712,-0.634123,0.844596,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,-0.234547,1.508734,-0.459859,0.483049,-0.230929,0.224952,1.056954,0.921730,-0.810815,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.751486,1.218393,0.231046,-0.756723,0.516833,0.275776,1.043127,-2.138114,0.232814,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


### Preprocessed Validation data (Feature) 만들기

In [8]:
# 아웃풋 경로 지정
transform_validation_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-validation-output')
# scikit_learn_inferencee_model 에서 Validation Transformer 생성
transformer_validation = scikit_learn_inferencee_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_validation_output_path,
    accept = 'text/csv')
# Preprocess validation input
transformer_validation.transform(s3_input_validation.config['DataSource']['S3DataSource']['S3Uri'], content_type='text/csv')
print('Waiting for transform job: ' + transformer_validation.latest_transform_job.job_name)
transformer_validation.wait()
preprocessed_validation_path = transformer_validation.output_path+transformer_validation.latest_transform_job.job_name
print(preprocessed_validation_path)



Attaching to tmpqdhq7j_3_algo-1-ii88h_1
algo-1-ii88h_1  | Processing /opt/ml/code
algo-1-ii88h_1  | Building wheels for collected packages: log-preprocessing
algo-1-ii88h_1  |   Building wheel for log-preprocessing (setup.py) ... done
algo-1-ii88h_1  |   Created wheel for log-preprocessing: filename=log_preprocessing-1.0.0-py2.py3-none-any.whl size=10220 sha256=ce64dbd4f772a1c96ff47b68cffffbe50d6f4eb2b176905913fee1b11e05b0f0
algo-1-ii88h_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-ahscs_nb/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-ii88h_1  | Successfully built log-preprocessing
algo-1-ii88h_1  | Installing collected packages: log-preprocessing
algo-1-ii88h_1  | Successfully installed log-preprocessing-1.0.0
algo-1-ii88h_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternat

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.


algo-1-ii88h_1  | 2020-08-13 08:38:35,829 INFO - root - Input_fn, Mode: feature_transform
algo-1-ii88h_1  | 2020-08-13 08:38:35,834 INFO - root - predict_fn, Mode: feature_transform
algo-1-ii88h_1  | After trainsformation
algo-1-ii88h_1  |   (0, 0)	-1.3722767476585274
algo-1-ii88h_1  |   (0, 1)	1.4361483643078248
algo-1-ii88h_1  |   (0, 2)	-0.7210321088903731
algo-1-ii88h_1  |   (0, 3)	-0.3104051644736047
algo-1-ii88h_1  |   (0, 4)	-0.6407980073546213
algo-1-ii88h_1  |   (0, 5)	0.4282497227369044
algo-1-ii88h_1  |   (0, 6)	-1.7359695510006319
algo-1-ii88h_1  |   (0, 7)	-0.8415698900657912
algo-1-ii88h_1  |   (0, 8)	-0.8827893216762703
algo-1-ii88h_1  |   (0, 9)	-1.0027868214581757
algo-1-ii88h_1  |   (0, 10)	-0.44388847702445833
algo-1-ii88h_1  |   (0, 12)	1.0
algo-1-ii88h_1  |   (0, 65)	1.0
algo-1-ii88h_1  |   (0, 68)	1.0
algo-1-ii88h_1  |   (1, 0)	-1.8020858585098782
algo-1-ii88h_1  |   (1, 1)	-0.5962380254245051
algo-1-ii88h_1  |   (1, 2)	1.2442493161458343
algo-1-ii88h_1  |   (1, 3

## PCA 모델 생성

In [9]:
import boto3
import sagemaker
from sagemaker import get_execution_role

ecr_namespace = 'sagemaker-training-containers/'
prefix = 'pca'

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)

057716757052
us-east-2
arn:aws:iam::057716757052:role/service-role/AmazonSageMaker-ExecutionRole-20191128T110038
sagemaker-us-east-2-057716757052


In [10]:
! cp pca_byoc_train.py docker/code/

In [11]:
%%writefile docker/Dockerfile

FROM 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
    
# install python package
RUN pip install joblib


ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

ENV PATH="/opt/ml/code:${PATH}"

# Copy training code
COPY code/* /opt/ml/code/
 
WORKDIR /opt/ml/code

# ENTRYPOINT ["python", "pca_train.py"]
# In order to use SageMaker Env varaibles, use the statement below
ENV SAGEMAKER_PROGRAM pca_byoc_train.py

Overwriting docker/Dockerfile


In [12]:
import os
os.environ['account_id'] = account_id
os.environ['region'] = region
os.environ['ecr_repository_name'] = ecr_repository_name

In [13]:
%%sh

ACCOUNT_ID=${account_id}
REGION=${region}
REPO_NAME=${ecr_repository_name}

echo $REGION
echo $ACCOUNT_ID
echo $REPO_NAME


# Get the login command from ECR in order to pull down the Tensorflow-gpu:1.5 image
$(aws ecr get-login --registry-ids 257758044811 --region ${region} --no-include-email)



docker build -f docker/Dockerfile -t $REPO_NAME docker

docker tag $REPO_NAME $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest

$(aws ecr get-login --no-include-email --registry-ids $ACCOUNT_ID)

aws ecr describe-repositories --repository-names $REPO_NAME || aws ecr create-repository --repository-name $REPO_NAME

docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest



us-east-2
057716757052
sagemaker-training-containers/pca
Login Succeeded
Sending build context to Docker daemon  10.24kB
Step 1/8 : FROM 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
 ---> 30adb1aa9af5
Step 2/8 : RUN pip install joblib
 ---> Using cache
 ---> 26bceed11435
Step 3/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> d2cbe65e931a
Step 4/8 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 3ed49376f0e2
Step 5/8 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 03be6787c4b2
Step 6/8 : COPY code/* /opt/ml/code/
 ---> b1ba50da3af1
Step 7/8 : WORKDIR /opt/ml/code
 ---> Running in a1e0f2b00ce6
Removing intermediate container a1e0f2b00ce6
 ---> e35443394139
Step 8/8 : ENV SAGEMAKER_PROGRAM pca_byoc_train.py
 ---> Running in ecd97c3880d7
Removing intermediate container ecd97c3880d7
 ---> 840f75c63caa
Successfully built 840f75c63caa
Successfully tagged sagemaker-training-containers/pca:latest
Login Succeeded
{
    "repositories

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [14]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

057716757052.dkr.ecr.us-east-2.amazonaws.com/sagemaker-training-containers/pca:latest


In [15]:
preprocessed_train_path_file = '{}/train.csv.out'.format(preprocessed_train_path)
preprocessed_validation_path_file = '{}/validation.csv.out'.format(preprocessed_validation_path)
print("preprocessed_train_path_file: \n", preprocessed_train_path_file)
print("preprocessed_validation_path_file: \n", preprocessed_validation_path_file)

preprocessed_train_path_file: 
 s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-13-08-38-2020-08-13-08-38-22-796/train.csv.out
preprocessed_validation_path_file: 
 s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-validation-output/sagemaker-scikit-learn-2020-08-13-08-38-2020-08-13-08-38-30-142/validation.csv.out


## PCA 학습

In [16]:
import pandas as pd

preprocessed_train_path_file = '{}/train.csv.out'.format(preprocessed_train_path)
pre_df = pd.read_csv(preprocessed_train_path_file, header=None)
print(pre_df.shape)
num_cols = pre_df.shape[1]
print("num_cols: ", num_cols)

(2333, 70)
num_cols:  70


In [17]:
import pandas as pd
# preprocessed_train_path_file = 's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-12-07-07-2020-08-12-07-07-08-229/train.csv.out'

churn_df = pd.read_csv(preprocessed_train_path_file, header=None)
churn_df.head()
train_y = churn_df.iloc[:,0]
train_X = churn_df.iloc[:,1:]

print("Shape of train_X: ", train_X.shape)
print("Shape of train_y: ", train_y.shape)

os.makedirs('./data', exist_ok =True)
np.savetxt('./data/churn-preprocessed.csv', train_X, delimiter=',',
           fmt='%1.5f'
          )

WORK_DIRECTORY = 'data'
prefix = 'Scikit-pca-custom'
train_input = sagemaker_session.upload_data(WORK_DIRECTORY,
                                            key_prefix="{}/{}".format(prefix, WORK_DIRECTORY)
                                           )
print("train_input: ", train_input)


'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.


Shape of train_X:  (2333, 69)
Shape of train_y:  (2333,)
train_input:  s3://sagemaker-us-east-2-057716757052/Scikit-pca-custom/data


In [18]:
%%time

import sagemaker

instance_type = 'local'
# instance_type = 'ml.m4.xlarge'

pca_estimator = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type= instance_type,
                                    base_job_name=prefix)

pca_estimator.set_hyperparameters(n_components= 15)

train_config = sagemaker.session.s3_input(train_input, content_type='text/csv')

pca_estimator.fit({'train': train_config})

Creating tmpch6034ao_algo-1-nq4c6_1 ... 
Attaching to tmpch6034ao_algo-1-nq4c6_12mdone
algo-1-nq4c6_1  | 2020-08-13 08:38:42,557 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-nq4c6_1  | 2020-08-13 08:38:42,559 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-nq4c6_1  | 2020-08-13 08:38:42,568 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-nq4c6_1  | 2020-08-13 08:38:42,568 sagemaker-containers INFO     Module pca_byoc_train does not provide a setup.py. 
algo-1-nq4c6_1  | Generating setup.py
algo-1-nq4c6_1  | 2020-08-13 08:38:42,568 sagemaker-containers INFO     Generating setup.cfg
algo-1-nq4c6_1  | 2020-08-13 08:38:42,569 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-nq4c6_1  | 2020-08-13 08:38:42,569 sagemaker-containers INFO     Installing module with the following command:
algo-1-nq4c6_1  | /miniconda3/bin/python -m pip install . 
algo-1-nq4c6_1  | Pr

tmpch6034ao_algo-1-nq4c6_1 exited with code 0
Aborting on container exit...
===== Job Complete =====
CPU times: user 158 ms, sys: 41.4 ms, total: 199 ms
Wall time: 4.16 s


# Transforming Train PCA

In [19]:
import pandas as pd

preprocessed_train_path_file = '{}/train.csv.out'.format(preprocessed_train_path)
pre_df = pd.read_csv(preprocessed_train_path_file, header=None)
print(pre_df.shape)
num_cols = pre_df.shape[1]
print("num_cols: ", num_cols)

(2333, 70)
num_cols:  70


In [20]:
instance_type = 'local'
# instance_type = 'ml.m4.2xlarge'
transform_train_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-pca-train-output')

pca_model = pca_estimator.create_model(
    env={'TRANSFORM_MODE': 'feature-transform', 'LENGTH_COLS': str(num_cols)})

# scikit_learn_inferencee_model 에서 Train Transformer 생성
transformer_train = pca_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_train_output_path,
    accept = 'text/csv')


# Preprocess training input
transformer_train.transform(preprocessed_train_path_file, 
                            content_type='text/csv',                            
                           )

print('Waiting for transform job: ' + transformer_train.latest_transform_job.job_name)
transformer_train.wait()

preprocessed_pca_train_path = transformer_train.output_path + transformer_train.latest_transform_job.job_name


Attaching to tmprog22g2y_algo-1-pfeu7_1
algo-1-pfeu7_1  | Processing /opt/ml/code
algo-1-pfeu7_1  | Building wheels for collected packages: pca-byoc-train
algo-1-pfeu7_1  |   Building wheel for pca-byoc-train (setup.py) ... done
algo-1-pfeu7_1  |   Created wheel for pca-byoc-train: filename=pca_byoc_train-1.0.0-py2.py3-none-any.whl size=8798 sha256=2e23c2bae581bb52414bea7edd047f8306a872facf898c82a2119b42926962ce
algo-1-pfeu7_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-yvpbgevm/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-pfeu7_1  | Successfully built pca-byoc-train
algo-1-pfeu7_1  | Installing collected packages: pca-byoc-train
algo-1-pfeu7_1  | Successfully installed pca-byoc-train-1.0.0
algo-1-pfeu7_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-pfeu7_

algo-1-pfeu7_1  | 172.18.0.1 - - [13/Aug/2020:08:38:51 +0000] "POST /invocations HTTP/1.1" 200 708583 "-" "-"
Gracefully stopping... (press Ctrl+C again to force)
Waiting for transform job: pca-2020-08-13-08-38-44-830-2020-08-13-08-38-44-830
.

In [21]:
print(preprocessed_pca_train_path)

s3://sagemaker-us-east-2-057716757052/Scikit-pca-custom/transformtrain-pca-train-output/pca-2020-08-13-08-38-44-830-2020-08-13-08-38-44-830


In [22]:
! aws s3 ls s3://sagemaker-us-east-2-057716757052/Scikit-pca-custom/transformtrain-pca-train-output/pca-2020-08-13-01-27-21-375-2020-08-13-01-27-21-375 --recursive

2020-08-13 01:27:28     707835 Scikit-pca-custom/transformtrain-pca-train-output/pca-2020-08-13-01-27-21-375-2020-08-13-01-27-21-375/train.csv.out.out


In [23]:
preprocessed_pca_train_path_file = '{}/train.csv.out.out'.format(preprocessed_pca_train_path)
pca_preoc_df = pd.read_csv(preprocessed_pca_train_path_file, header=None)
pca_preoc_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,-0.823085,-0.108811,0.452843,-0.030507,-1.031997,-2.956747,-0.051300,-0.706446,1.064536,-0.437290,-0.507988,-0.181687,-0.081850,-0.027787,-0.016994
1,0.0,-0.343474,0.091423,1.949026,1.268235,0.009081,0.421556,-0.988975,0.868835,-0.542355,-0.341819,0.825626,-0.158788,-0.041426,-0.079993,0.085742
2,1.0,-0.764309,0.011604,0.822738,-1.429299,-1.627301,-0.741560,0.550952,0.428517,-0.326231,-0.289266,-1.686701,-0.128045,-0.027735,-0.038399,-0.175243
3,0.0,-0.825983,-0.722031,-0.339858,-0.980679,-0.260783,0.357790,-0.669738,-1.122771,-1.451326,-0.006193,-1.091866,-0.134262,-0.031474,0.000403,-0.028218
4,0.0,1.830756,0.701878,0.194737,-1.351443,-0.729668,0.845136,0.150495,0.084293,0.180299,0.751587,0.435199,-0.075148,-0.031001,0.018728,-0.007071


## PCA Validation Transforming

In [24]:
preprocessed_validation_path

's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-validation-output/sagemaker-scikit-learn-2020-08-13-08-38-2020-08-13-08-38-30-142'

In [25]:
import pandas as pd

preprocessed_validation_path_file = '{}/validation.csv.out'.format(preprocessed_validation_path)
pre_df = pd.read_csv(preprocessed_validation_path_file, header=None)
print(pre_df.shape)
num_cols = pre_df.shape[1]
print("num_cols: ", num_cols)

(666, 70)
num_cols:  70


In [26]:


instance_type = 'local'
# instance_type = 'ml.m4.2xlarge'
transform_validation_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-pca-validation-output')

pca_model = pca_estimator.create_model(
    env={'TRANSFORM_MODE': 'feature-transform', 'LENGTH_COLS': str(num_cols)})

# scikit_learn_inferencee_model 에서 Train Transformer 생성
transformer_validation = pca_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_validation_output_path,
    accept = 'text/csv')


# Preprocess training input
transformer_validation.transform(preprocessed_validation_path_file, 
                            content_type='text/csv',                            
                           )

print('Waiting for transform job: ' + transformer_validation.latest_transform_job.job_name)
transformer_validation.wait()

preprocessed_pca_validation_path = transformer_validation.output_path + transformer_validation.latest_transform_job.job_name
print(preprocessed_pca_validation_path)

Attaching to tmprdiuhqqs_algo-1-az4eb_1
algo-1-az4eb_1  | Processing /opt/ml/code
algo-1-az4eb_1  | Building wheels for collected packages: pca-byoc-train
algo-1-az4eb_1  |   Building wheel for pca-byoc-train (setup.py) ... done
algo-1-az4eb_1  |   Created wheel for pca-byoc-train: filename=pca_byoc_train-1.0.0-py2.py3-none-any.whl size=8800 sha256=6cd0d9213244be1ce6f7fc863a4f9c0f8c6eb91986a1cfc0d79e4a1a3e681e77
algo-1-az4eb_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-dvwxm_a9/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-az4eb_1  | Successfully built pca-byoc-train
algo-1-az4eb_1  | Installing collected packages: pca-byoc-train
algo-1-az4eb_1  | Successfully installed pca-byoc-train-1.0.0
algo-1-az4eb_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-az4eb_

algo-1-az4eb_1  |       0   ...   69
algo-1-az4eb_1  | 0  0.0  ...  1.0
algo-1-az4eb_1  | 1  0.0  ...  0.0
algo-1-az4eb_1  | 
algo-1-az4eb_1  | [2 rows x 70 columns]
algo-1-az4eb_1  | os.getenv('LENGTH_COLS') :  70
algo-1-az4eb_1  | type: os.getenv('LENGTH_COLS'):  <class 'str'>
algo-1-az4eb_1  | type of components:  <class 'numpy.ndarray'>
algo-1-az4eb_1  | shape of components:  (666, 15)
algo-1-az4eb_1  | 2020-08-13 08:38:58,906 INFO - root - predict_fn: PCA components: 
algo-1-az4eb_1  | '[[ 0.00000000e+00  1.64752263e+00  1.32131166e+00 ... -1.03801909e-01
algo-1-az4eb_1  |    6.41176159e-03 -8.10089247e-02]
algo-1-az4eb_1  |  [ 0.00000000e+00 -5.68755661e-01  2.09781517e-01 ... -6.18656868e-02
algo-1-az4eb_1  |   -4.44238892e-03 -1.16230329e-02]
algo-1-az4eb_1  |  [ 0.00000000e+00  1.85617085e+00 -5.58974510e-01 ... -4.39454349e-03
algo-1-az4eb_1  |    7.40018485e-02 -5.84865225e-02]
algo-1-az4eb_1  |  ...
algo-1-az4eb_1  |  [ 0.00000000e+00  1.82761578e+00 -1.00779899e+00 ... -1.

In [27]:
preprocessed_pca_validation_path_file = '{}/validation.csv.out.out'.format(preprocessed_pca_validation_path)
pca_val_preoc_df = pd.read_csv(preprocessed_pca_validation_path_file, header=None)
pca_val_preoc_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,1.647523,1.321312,-0.096829,0.255327,1.185455,-1.356901,-0.144986,-1.376000,0.192108,-0.407086,1.423016,-0.110445,-0.103802,0.006412,-0.081009
1,0.0,-0.568756,0.209782,0.928801,-0.421777,-1.249794,-1.183297,-2.058817,0.980869,-1.320562,-1.227984,0.184400,-0.143090,-0.061866,-0.004442,-0.011623
2,0.0,1.856171,-0.558975,-1.969703,0.315529,0.132519,-0.136767,-0.623570,0.832536,1.131302,1.985652,-0.235460,-0.172427,-0.004395,0.074002,-0.058487
3,0.0,-0.681862,-1.326923,-0.849271,-1.372694,-0.960617,1.216868,-0.801562,2.278247,-0.030436,0.099985,-1.289597,-0.164498,-0.015569,0.015632,0.022708
4,0.0,2.290559,0.265566,-0.891170,0.860266,0.113337,0.457285,-0.131537,-0.713534,-0.735986,0.545870,1.292899,-0.146049,0.979650,0.010722,0.002913


## Inference Pipeline <a class="anchor" id="pipeline_setup"></a>

아래 그림과 같이 위에서 생성한 전처리, 알고리즘 학습, 후처리의 세가지 모델을 가지고 1개의 단일 모델을 만들어 Inference Pipleline을 생성 합니다. <br>
**입력 데이타 가공이 없이 실제 데이타가 입력이 되면, 1개의 단일 모델을 통해서 최종적으로 예측 결과인 True, False의 결과 값이 제공 됩니다.**

![Inference-pipeline](img/Fig2.2.inference_pipeline.png)


**Machine Learning Model Pipeline (Inference Pipeline)는 create_model() 를 호출하여 만들 수 있습니다.** <br>
예를 들어 여기서는 the fitted Scikit-learn inference model, the fitted Xgboost model and the psotprocessing model 의 세가지 모델을 가지고 만듦니다.

아래는 세개 모델을 생성함. 전처리, 후처리 모델 생성시에는 환경 변수를 제공 함

In [28]:
# 전처리 모델
scikit_learn_pre_process_model = sklearn_preprocessor.create_model(
    env={'TRANSFORM_MODE': 'feature-transform'})    

# PCA 전처리 모델
pca_infer_model = pca_estimator.create_model(
    env={'TRANSFORM_MODE': 'inverse-label-transform', 'LENGTH_COLS': '69'})


In [29]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3

from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = 'churn-model-inference-pipeline-' + timestamp_prefix


sklearn_preprocessor.create_model

pipeline_model = PipelineModel(
    name = model_name,
    role = role,
    models = [
        scikit_learn_pre_process_model,
        pca_infer_model,        
    ]
)

In [30]:
%%time

instance_type='ml.t2.medium'
# instance_type='local'
endpoint_name= 'churn-model-pipeline-endpoint-' + timestamp_prefix

deployed_model = pipeline_model.deploy(
    initial_instance_count=1, 
    instance_type= instance_type, 
    endpoint_name = endpoint_name,        
    wait = True
)

---------------------!CPU times: user 322 ms, sys: 21.3 ms, total: 344 ms
Wall time: 10min 32s


In [34]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
import sagemaker
sagemaker_session = sagemaker.Session()

predictor = RealTimePredictor(
    endpoint = endpoint_name,
    sagemaker_session = sagemaker_session,
    serializer = csv_serializer,
    content_type = CONTENT_TYPE_CSV,
    accept = CONTENT_TYPE_JSON
)



In [35]:

def make_inference_format(sample):
    instance = str()
    for i, token in enumerate(sample):
        # print(token)
        if i > 0:
            instance = instance  + ',' + str(token) 
        else:
            instance = instance  +  str(token) 
    return instance


In [36]:
test_df = pd.read_csv("churn_data/batch_transform_test.csv", header=None)

for i in range(10):
    sample = test_df.iloc[i,:]
    instance = make_inference_format(sample)
    print("instance: \n", instance)

    payload = instance
    churn_result = predictor.predict(payload)
    print("Churn result?: \n", churn_result)
    print("")
    break

instance: 
 KS,186,510,400-6454,no,no,0,137.8,97,23.43,187.7,118,15.95,146.4,85,6.59,8.7,6,2.35,1
Churn result?: 
 b'[[-0.8904540421006801, -0.9053302434571645, -1.1207797695964323, 0.362526924252823, 0.38380596846260495, 0.20507417633945163, 1.6356116093943496, -1.6222531924663215, 0.7557514550972801, -0.4338814171841123, -0.2936240682721869, -0.164310887134048, -0.03178033677512544, -0.08304269602916978, 0.030578711404020565]]'

